# Detecção de objetos em movimento

Para exemplificar uma aplicação de visão computacional em vídeo, vamos realizar a detecção de objetos em movimento (ou regiões em que há movimento) em vídeo.

O exercício envolverá a implementação de dois métodos que serão usados em meio ao processo de detecção de movimento.     



- **1) Thresholding**: Esse método envolve produzir um imagem que contenha apenas duas cores (com intensidades iguais), geralmente preto e branco, a partir de um threshold (limiar) pré-definido. Abaixo temos um exemplo de como essa função será usada em nosso contexto
    
<img src="https://raw.githubusercontent.com/lab-cameras-each/cv-workshop-live/master/recursos/exemplo_thresholding.png" alt="thresh" width="1400"/>
    
    
    
- **2) Desenho de retângulos**: Esse método deverá, a partir de uma lista de retângulos (definidos por suas coordenadas), desenhá-los na imagem de sáida juntamente com um texto informando o número total de retângulos 


<img src="https://raw.githubusercontent.com/lab-cameras-each/cv-workshop-live/master/recursos/retangulos.png" alt="ret" width="500" />
    
    
    
    
    
    
#### Os detalhes de implemetanção de cada um dos métodos se encontram ao decorrer deste notebook, no momento em que são necessários.    

    
    

    

In [ ]:
# Importação de bibliotecas necessárias
import cv2
import sys
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
import requests
from IPython.display import HTML
from base64 import b64encode
from google.colab.patches import cv2_imshow    

In [ ]:
# Download de vídeo
url = 'https://github.com/lab-cameras-each/cv-workshop-live/blob/master/recursos/campus_video.mp4?raw=true'
r = requests.get(url, allow_redirects=True)

In [ ]:

# Exibir vídeo
data_url = "data:video/mp4;base64," + b64encode(r.content).decode()
HTML(f"""
<video width=800 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
# Salvar vídeo em arquivo
open('video_original.mp4', 'wb').write(r.content)


In [ ]:
#Armazenar cada frame do video em uma lista
cap = cv2.VideoCapture('video_original.mp4')
frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
  
    if ret == True:
        frames.append(frame)
    else: 
        break

cap.release()

In [ ]:
#Número de frames
len(frames)

In [ ]:
#Resolução
frames[0].shape

## Detecção de movimento

Nas próximas interações, faremos a execução passo-a-passo do algoritmo de detecção de movimento

Primeiramente vamos detectar as regioes de movimento entre dois frames do vídeo: o quadro 200 e 201

In [ ]:
#Exibindo o Frame na posição 200
cv2_imshow(frames[200])

In [ ]:
weight = 0.6 # Peso dos novos frames na atualização de backgorund
avg = None

In [ ]:
frame_copy = frames[200].copy() 
frame_gray = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2GRAY) # Transformação do frame para escala de cinza

In [ ]:
cv2_imshow(frame_gray)

In [ ]:
# Inicializa background
avg = frame_gray.copy().astype("float")

In [ ]:
#Atualiza a média da imagem de fundo, precisa ser feito no caso de o pano de fundo ser móvel
#https://docs.opencv.org/3.4/d7/df3/group__imgproc__motion.html
# avg(x,y) = (1−weight)⋅avg(x,y) + weight⋅frame_gray(x,y)
cv2.accumulateWeighted(frame_gray, avg, weight)

In [ ]:
cv2_imshow(avg)

In [ ]:
# Diferença em relação a proximo frame
frame_copy = frames[201].copy()
frame_gray = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2GRAY)

frameDelta = cv2.absdiff(frame_gray, cv2.convertScaleAbs(avg)) # convertScaleAbs conversao para inteiros

In [ ]:
cv2_imshow(frameDelta)

In [ ]:
#Atualizar média
cv2.accumulateWeighted(frame_gray, avg, weight)

### Exercício 1 - Função de thresholding
Agora, você deverá implementar a função de threshold conforme a instrução a seguir

```python

'''
apply_threshold: Método de thresholding

Se pixel > threshold, entao assume o valor max_value
Se pixel <= threshold, entao assume o valor 0


Parâmetros:
  image (np.array) - Imagem na qual será aplicada a operação
  threshold (int) - Valor do limiar para aplicação de thresholding
  max_value (int) - Valor máximo que píxel deve assumir se maior que threshold

Retorna:
  new_image (np.array) - Imagem após aplicação de thresholding

'''

def apply_threshold(image, threshold, max_value):
    new_image = image.copy()
    #Aplicação de thresholding
    #.
    #.
    return new_image
```

In [ ]:
#Aplicar threshold
thresh = apply_threshold(frameDelta, threshold=5, max_value=255)

In [ ]:
cv2_imshow(thresh)

In [ ]:
# Encontrando contornos na imagem, ou seja, regiões com aglomerações de píxeis.
# No nosso contexto, consideramos que são objetos

#Fonte https://github.com/opencv/opencv/blob/master/modules/imgproc/src/contours.cpp
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
len(contours)

In [ ]:
min_area = 400 # Area minima para considerar um contorno como um objeto

bounding_boxes = []
for c in contours:
    
    if cv2.contourArea(c) < min_area:
        continue
        
    bounding_boxes.append(cv2.boundingRect(c))

### Exercício 2 - Função para desenhar retângulos e texto
Agora, você deverá implementar o método que irá desenhar retângulos ao redor dos objetos

```python
'''
draw_info: Método para inserir desenhos dos retângulos e informação de quantidade desses em um frame

Parâmetros:
  frame (np.array) - Imagem na qual será aplicada a operação
  bounding_boxes (list) - Bounding boxes a serem desenhadas

Retorna:
  frame_draw (np.array) - Imagem após desenho de retângulos

'''
def draw_info(frame, bounding_boxes):
    frame_draw = frame.copy()

    # Para cada retângulo em bounding_boxes, inserir seu desenho em frame_draw
    #.
    #.
    # Inserir texto informando quantidade de retângulos (objetos)   
    #.
    #.
        
    return frame_draw
```

In [ ]:
# Desenho de retângulos e texto
frame_draw = draw_info(frames[201], bounding_boxes)

In [ ]:
cv2_imshow(frame_draw)

In [ ]:
# Função final. 
# Essa função já esta definida e utiliza os métodos que você implementou anteriormente

# A função detecta objetos em movimento considerando o frame passado como parametro e a média (avg), desenhado os retângulos na cena
def detect_movement_frame(frame, avg, weight=0.6, min_area=400):
    frame_copy = frame.copy()
    frame_gray = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2GRAY)

    if avg is None:
        avg = frame_gray.copy().astype("float")

    frameDelta = cv2.absdiff(frame_gray, cv2.convertScaleAbs(avg))
    cv2.accumulateWeighted(frame_gray, avg, weight)

    thresh = apply_threshold(frameDelta, threshold=5, max_value=255) # Implementar

    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_boxes = []
    for c in contours:
        # Ignorar se menor que área minima
        if cv2.contourArea(c) < min_area:
            continue
        bounding_boxes.append(cv2.boundingRect(c))
        
        
    return bounding_boxes, avg

In [ ]:
# Aplicar detecção de objetos em movimento para todos os frames do vídeo

cap = cv2.VideoCapture('video_original.mp4') # Abir vídeo
frame_width = int(cap.get(3)) # Obtém largura do vídeo para usar no arquivo de sáida
frame_height = int(cap.get(4))# Obtém altura do vídeo para usar no arquivo de sáida

# Arquivo de vídeo de saída
out = cv2.VideoWriter('video_processado.avi',cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width,frame_height))

avg = None
while(cap.isOpened()): # Enquanto vídeo não chegar ao fim
    ret, frame = cap.read() # Lê próximo frame
    
    if ret == True: # Se leitura feita com sucesso
        bounding_boxes, avg = detect_movement_frame(frame, avg, weight=0.6, min_area=800) # Detecta objetos em movimento e os desenha     
        if len(bounding_boxes) > 0: # Se há objetos
             frame = draw_info(frame, bounding_boxes) # Desenha retângulos correspondentes

        out.write(frame) # Escreve frame no aquivo de vídeo de sáida
    else: 
        break

cap.release() # Fecha vídeo de entrada

In [ ]:

!ffmpeg -i video_processado.avi video_processado.mp4 -y # Conversão de avi para mp4 (suportando por google colab)

In [ ]:
# Exibição do vídeo final

f = open('video_processado.mp4', 'rb')
video_processado = f.read()

data_url = "data:video/mp4;base64," + b64encode(video_processado).decode()
HTML(f"""
<video width=800 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

Para ir além:
- Rodar localmente (Jupyter Notebook)
- Mostrar vídeo enquanto faz a leitura
- Tentar fazer adaptação para rodar localmente usando webcam